In [ ]:
# 导入常用包
%xmode Plain
%matplotlib inline
import pandas as pd
import pymysql
import matplotlib.pyplot as plt
from matplotlib.pylab import style
style.use ('ggplot')
plt.rcParams['font.sans-serif'] = ['Heiti TC']
plt.rcParams['font.serif'] = ['Heiti TC']
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# 设置从数据库获取表格的函数
host = "121.37.185.35"
port = 8306
user = "root"
password = "root"
database = "bilibili"

conn = pymysql.connect (
    host=host,
    user=user,
    password=password,
    port=port,
    database=database
)

def get_table_content(table_name,index_col=None):
    sql = f'select * from {table_name}'
    try:
        df = pd.read_sql (sql, con=conn,index_col=index_col)
    except Exception as e:
        print(e)
        return False
    return df

In [ ]:
# 获取所有表格数据
vinfo = get_table_content("vinfo","aid")
aid_hotword = get_table_content("aid_hotword","aid")
aid_tag = get_table_content("aid_tag")
follower = get_table_content("follower")
hotword = get_table_content("hotword")
userinfo = get_table_content("userinfo","uid")
vresult = get_table_content("vresult")

In [ ]:
vresult_list = {}
tmpresult = pd.DataFrame(columns=vresult.columns)
index = 1

aids = vresult["aid"].sort_values().drop_duplicates()
for aid in aids:
    vresult_list[aid] = vresult[vresult["aid"].apply(lambda x:x == aid)]


In [ ]:
vresult.iloc[:,1:-1]

In [41]:
num = 0
info_num = 0
minimum = 9999
maximum = 0
formed_vresult_list = {} # 存储可以用于构建序列的信息
for aid,ob in vresult_list.items():
    if len(ob)>5:
        num += 1
        info_num += len(ob)
        minimum = min(len(ob),minimum)
        maximum = max(len(ob),maximum)
        cre_time = vinfo.loc[aid]["cre_time"]
        for i,row in ob.iterrows():
            alive_day = row["timestamp"] - cre_time
            alive_day = alive_day.days
            alive_weekday = row["timestamp"].weekday()
            cre_weekday = cre_time.weekday()
            ob.loc[i,"alive_day"] = alive_day
            ob.loc[i,"alive_weekday"] = alive_weekday
            ob.loc[i,"cre_weekday"] = cre_weekday
            ob.loc[i,"cre_time"] = cre_time
        ob.index = ob["timestamp"]
        ob = ob.iloc[:,1:-1]
        resample = ob.resample("6H").mean()
        break
        formed_vresult_list[aid] = ob
print(f"总共结果记录视频条数：{len(vresult_list)}，其中有效记录超过5条的记录有{num},"
      f"其中最少条数{minimum}，最多条数{maximum},总共信息记录有{info_num}")

总共结果记录视频条数：2816，其中有效记录超过5条的记录有1,其中最少条数20，最多条数20,总共信息记录有20


/Users/hyacinth/PycharmProjects/bili_analyze/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [ ]:
for i in vresult_list.values():
    columns = i.columns
    break
merge_vresult = pd.DataFrame(index=range(info_num),columns=columns)
i = 0
for li in vresult_list.values():
    for k,row in li.iterrows():
        merge_vresult.iloc[i] = row
        i += 1

In [ ]:
merge_vresult.to_csv("../storage/vresult.csv")